Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
data=pd.read_csv('/content/drive/MyDrive/air/train_xyz.csv')
data

,x,y,z,target
0,1.890000,2.230000,7.210000,0
1,2.030000,2.150000,7.140000,0
2,2.140000,2.330000,6.760000,0
3,2.330000,2.700000,6.680000,0
4,2.450000,2.730000,6.160000,0
...,...,...,...,...
406195,0.198186,-19.142152,15.008114,3
406196,-10.358698,1.879607,14.794481,3
406197,2.820141,-7.553847,-7.744405,3
406198,-15.877624,-7.054193,-0.982989,3


In [ ]:
df=pd.DataFrame(data)
df.columns=['x-axis','y-axis','z-axis','number']
df

,x-axis,y-axis,z-axis,number
0,1.890000,2.230000,7.210000,0
1,2.030000,2.150000,7.140000,0
2,2.140000,2.330000,6.760000,0
3,2.330000,2.700000,6.680000,0
4,2.450000,2.730000,6.160000,0
...,...,...,...,...
406195,0.198186,-19.142152,15.008114,3
406196,-10.358698,1.879607,14.794481,3
406197,2.820141,-7.553847,-7.744405,3
406198,-15.877624,-7.054193,-0.982989,3


In [ ]:
df['number'].value_counts()

44    13315
3     13300
8     13296
2     13280
5     13279
0     13270
6     13257
7     13243
1     13233
9     13230
Name: number, dtype: int64

In [ ]:
df['number'] = df['number'].replace(44,4)
df['number'].value_counts()


4    13315
3    13300
8    13296
2    13280
5    13279
0    13270
6    13257
7    13243
1    13233
9    13230
Name: number, dtype: int64

## Standardized data

In [ ]:
from sklearn.preprocessing import StandardScaler
x=df[['x-axis','y-axis','z-axis']]
y=df['number']
scaler=StandardScaler()
X=scaler.fit_transform(x)


In [ ]:
scaled_X=pd.DataFrame(data=X,columns=['x-axis','y-axis','z-axis'])
scaled_X['number']=y.values

scaled_X

,x-axis,y-axis,z-axis,number
0,-0.000241,0.009040,-0.010953,0
1,0.001313,0.008173,-0.011701,0
2,0.002534,0.010124,-0.015760,0
3,0.004643,0.014135,-0.016615,0
4,0.005974,0.014461,-0.022170,0
...,...,...,...,...
406195,-0.019016,-0.222647,0.072353,3
406196,-0.136175,0.005242,0.070070,3
406197,0.010082,-0.097022,-0.170707,3
406198,-0.197424,-0.091606,-0.098477,3


In [ ]:
df

,x-axis,y-axis,z-axis,number
0,1.890000,2.230000,7.210000,0
1,2.030000,2.150000,7.140000,0
2,2.140000,2.330000,6.760000,0
3,2.330000,2.700000,6.680000,0
4,2.450000,2.730000,6.160000,0
...,...,...,...,...
406195,0.198186,-19.142152,15.008114,3
406196,-10.358698,1.879607,14.794481,3
406197,2.820141,-7.553847,-7.744405,3
406198,-15.877624,-7.054193,-0.982989,3


In [ ]:
import scipy.stats as stats

In [ ]:
fs=20
frame_size=fs*9
hop_size=fs*2

In [ ]:
def get_frames(df,frame_size,hop_size):
    N_FEATURES=3
    
    frames=[]
    lables=[]
    for i in range(0,len(df)-frame_size,hop_size):
        x=df['x-axis'].values[i: i+frame_size]
        y=df['y-axis'].values[i: i+frame_size]
        z=df['z-axis'].values[i: i+frame_size]   
        
        lable = stats.mode(df['number'][i: i+frame_size])[0][0]
        frames.append([x,y,z])
        lables.append(lable)
    frames=np.asarray(frames).reshape(-1,frame_size,N_FEATURES)
    lables=np.asarray(lables)
    
    return frames ,lables

In [ ]:
X_x,Y_y=get_frames(df,frame_size,hop_size)

In [ ]:
X_x.shape , Y_y.shape

((10151, 180, 3), (10151,))

In [ ]:
from sklearn.model_selection import KFold, cross_val_score, train_test_split


In [ ]:
X_train , X_test, y_train, y_test = train_test_split(X_x,Y_y,test_size=0.2,random_state=0,stratify=Y_y)

In [ ]:
X_test.shape,X_train.shape,y_test.shape,y_train.shape

((2031, 180, 3), (8120, 180, 3), (2031,), (8120,))

In [ ]:
X_test= X_test.reshape(2031, 180, 3,1)
X_train=X_train.reshape(8120, 180, 3, 1)

In [ ]:
X_test.shape,X_train.shape

((2031, 180, 3, 1), (8120, 180, 3, 1))

In [ ]:
X_test

### 2D CNN Model

In [ ]:
from keras.models import Sequential
from tensorflow.keras.layers import Conv2D ,Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam , RMSprop

In [ ]:
model=Sequential()
model.add(Conv2D(16, (2,2), activation = 'relu', input_shape= X_train[0].shape))
model.add(Dropout(0.1))

model.add(Conv2D(32, (2,2), activation = 'relu'))
model.add(Dropout(0.2))

model.add(Flatten())

model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(4, activation='softmax'))

In [ ]:
model.compile(optimizer=Adam(learning_rate = 0.001), loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs = 100, batch_size=2, validation_data= (X_test, y_test),verbose=1)

Epoch 1/100
4060/4060 [==============================] - 25s 6ms/step - loss: 0.7778 - accuracy: 0.7365 - val_loss: 0.1370 - val_accuracy: 0.9542
Epoch 2/100
4060/4060 [==============================] - 25s 6ms/step - loss: 0.2167 - accuracy: 0.9209 - val_loss: 0.0849 - val_accuracy: 0.9749
Epoch 3/100
4060/4060 [==============================] - 24s 6ms/step - loss: 0.1592 - accuracy: 0.9502 - val_loss: 0.0754 - val_accuracy: 0.9759
Epoch 4/100
4060/4060 [==============================] - 24s 6ms/step - loss: 0.1777 - accuracy: 0.9491 - val_loss: 0.0712 - val_accuracy: 0.9774
Epoch 5/100
4060/4060 [==============================] - 24s 6ms/step - loss: 0.1194 - accuracy: 0.9615 - val_loss: 0.0439 - val_accuracy: 0.9833
Epoch 6/100
4060/4060 [==============================] - 23s 6ms/step - loss: 0.1472 - accuracy: 0.9605 - val_loss: 0.0681 - val_accuracy: 0.9778
Epoch 7/100
4060/4060 [==============================] - 23s 6ms/step - loss: 0.1555 - accuracy: 0.9651 - val_loss: 0.0814 -

KeyboardInterrupt: ignored